# `nanoGPT`: GPT-2 Small (125M Params)

## Install / Setup

### First Time Running

We need to install `ngpt` and setup the Shakespeare dataset

This will need to be ran the first time you are running this notebook.

Following the

```python
!python3 -m pip install nanoGPT
```

you will need to restart your runtime (Runtime -> Restart runtime)

After this, you should be able to

```python
>>> import ngpt
>>> ngpt.__file__
'/content/nanoGPT/src/ngpt/__init__.py'
```

In [1]:
%%bash

python3 -c 'import ngpt; print(ngpt.__file__)' 2> '/dev/null'

if [[ $? -eq 0 ]]; then
    echo "Has ngpt installed. Nothing to do."
else
    echo "Does not have ngpt installed. Installing..."
    git clone 'https://github.com/saforem2/nanoGPT'
    python3 nanoGPT/data/shakespeare_char/prepare.py
    python3 -m pip install -e nanoGPT -vvv
fi

/lus/grand/projects/datascience/foremans/locations/thetaGPU/projects/saforem2/nanoGPT/src/ngpt/__init__.py
Has ngpt installed. Nothing to do.


## Post Install

If installed correctly, you should be able to:

```python
>>> import ngpt
>>> ngpt.__file__
'/path/to/nanoGPT/src/ngpt/__init__.py'
```

In [2]:
%load_ext autoreload
%autoreload 2

import ngpt
from enrich import get_logger
log = get_logger('jupyter')
log.info(ngpt.__file__)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
[2023-11-30 07:22:57][INFO][3434626787.py:7] - /lus/grand/projects/datascience/foremans/locations/thetaGPU/projects/saforem2/nanoGPT/src/ngpt/__init__.py


## Build Trainer

Explicitly, we:

1. `setup_torch(...)`
2. Build `cfg: DictConfig = get_config(...)`
3. Instnatiate `config: ExperimentConfig = instantiate(cfg)`
4. Build `trainer = Trainer(config)`

In [3]:
import os
from ezpz import setup_torch
from hydra.utils import instantiate
from ngpt.configs import get_config
from ngpt.trainer import Trainer

os.environ['MASTER_PORT'] = '4235'
rank = setup_torch('DDP', seed=1234)
cfg = get_config(
    [
        'data=owt',              # open web text
        'model=gpt2',            # gpt2 arch.
        'optimizer=gpt2',
        'train=gpt2',
        'train.init_from=gpt2',  # init from GPT2
        'train.max_iters=1000',
        'train.dtype=bfloat16',
    ]
)
config = instantiate(cfg)
trainer = Trainer(config)

--------------------------------------------------------------------------

  Local host:   thetagpu23
  Local device: mlx5_0
--------------------------------------------------------------------------


[2023-11-30 07:23:11][INFO][configs.py:72] - Setting HF_DATASETS_CACHE to /lus/grand/projects/datascience/foremans/locations/thetaGPU/projects/saforem2/nanoGPT/.cache/huggingface/datasets
Failed to download font: Source Sans Pro, skipping!
Failed to download font: Titillium WebRoboto Condensed, skipping!
[2023-11-30 07:23:15][INFO][configs.py:295] - Loading val from /lus/grand/projects/datascience/foremans/locations/thetaGPU/projects/saforem2/nanoGPT/data/openwebtext/val.bin
[2023-11-30 07:23:15][INFO][configs.py:295] - Loading train from /lus/grand/projects/datascience/foremans/locations/thetaGPU/projects/saforem2/nanoGPT/data/openwebtext/train.bin
[2023-11-30 07:23:15][WARNING][configs.py:330] - No meta.pkl found, assuming GPT-2 encodings...
[2023-11-30 07:23:15][INFO][configs.py:270] - Rescaling GAS -> GAS // WORLD_SIZE = 1 // 1
[2023-11-30 07:23:15][INFO][configs.py:432] - Tokens per iteration: 12,288
[2023-11-30 07:23:15][INFO][configs.py:454] - Using <torch.amp.autocast_mode.auto

In [4]:
query = "What is a supercomputer?"
outputs = trainer.evaluate(query, num_samples=1, display=False)
log.info("['prompt']: '{query}'")
log.info("['response']:\n\n" + fr"{outputs['0']['raw']}")

[2023-11-30 07:23:33][INFO][1657463709.py:3] - ['prompt']: '{query}'
[2023-11-30 07:23:33][INFO][1657463709.py:4] - ['response']:

What is a supercomputer? When did you first learn it?

I used to work in an Apple Computer. It was called the "Elgin" Computer. It was the first computer that I had seen on TV. I went to college in 1983. I was at Arizona State University and I studied computer science. I later joined a computer science program at MIT. But my first computer was the Intel Core. It was from 1986. I went to MIT where I got my PhD and was at the lab. When I did graduate school in 1987, I went to Stanford where I made a few jobs.

Did you ever think of programming as the "new media?" Did you ever think of learning about computers as the "new medium?"

The world was changing. I started to think about the Internet as a new medium. I thought of computers as different from television and movies. My research led me to the Computer World. Where did you get that idea? It was a computer 

## Train Model

Legend:

<div style="text-align:left;">

|  **NAME**  |     **DESCRIPTION**          |
|:----------:|:----------------------------:|
|   `step`   | Current training step        |
|   `loss`   | Loss value                   |
|   `dt`     | Time per step (in **ms**)    |
|   `sps`    | Samples per second           |
|   `mtps`   | (million) Tokens per sec     |
|   `mfu`    | Model Flops Utilization*     |

*in units of A100 `bfloat16` peak FLOPS

</div>

In [5]:
trainer.train(train_iters=1000)

  0%|          | 0/1000 [00:00<?, ?it/s]

[2023-11-30 07:24:29][INFO][trainer.py:518] - step=100 loss=3.119 dt=267.101 sps=3.744 mtps=0.046 mfu=12.610 train_loss=3.114 val_loss=3.108
[2023-11-30 07:24:56][INFO][trainer.py:518] - step=200 loss=2.961 dt=228.076 sps=4.385 mtps=0.054 mfu=12.825 train_loss=3.114 val_loss=3.108
[2023-11-30 07:25:22][INFO][trainer.py:518] - step=300 loss=3.022 dt=276.843 sps=3.612 mtps=0.044 mfu=12.759 train_loss=3.114 val_loss=3.108
[2023-11-30 07:25:48][INFO][trainer.py:518] - step=400 loss=3.002 dt=234.589 sps=4.263 mtps=0.052 mfu=12.919 train_loss=3.114 val_loss=3.108
[2023-11-30 07:26:14][INFO][trainer.py:518] - step=500 loss=3.004 dt=291.194 sps=3.434 mtps=0.042 mfu=12.784 train_loss=3.114 val_loss=3.108
[2023-11-30 07:26:41][INFO][trainer.py:518] - step=600 loss=3.071 dt=267.192 sps=3.743 mtps=0.046 mfu=12.766 train_loss=3.114 val_loss=3.108
[2023-11-30 07:27:08][INFO][trainer.py:518] - step=700 loss=3.118 dt=247.305 sps=4.044 mtps=0.050 mfu=12.851 train_loss=3.114 val_loss=3.108
[2023-11-30 0

## Evaluate Model

In [6]:
query = "What is a supercomputer?"
outputs = trainer.evaluate(query, num_samples=1, display=False)
log.info("['prompt']: '{query}'")
log.info("['response']:\n\n" + fr"{outputs['0']['raw']}")

[2023-11-30 07:28:34][INFO][1657463709.py:3] - ['prompt']: '{query}'
[2023-11-30 07:28:34][INFO][1657463709.py:4] - ['response']:

What is a supercomputer? A supercomputer is a machine that is capable of using any available computing talent while being clever. It is the most beautiful piece of technology on the planet. In comparison to today's computers, the 'supercomputer' is a small piece of technology that does not require any knowledge of maths, physics or mathematics of the material world.

The computer is not just a computer that is smart - it is a computing device that has a deep understanding of the meaning and reality of the world around it - and it is an all-purpose machine, like a toy. When it is not 'computer' it is a robot.

This is a list of the wonderful machines on the planet.

The 'computer' itself is a very powerful machine. It is a very powerful computer at its best - it is capable of handling all sorts of things - many things, including playing video games, social m